In [3]:
!pip install -q "smolagents[litellm]" python-dotenv duckduckgo-search

In [2]:
import os
from dotenv import load_dotenv

# .env 파일을 만들어 OPENAI_API_KEY="sk-..." 형식으로 키를 저장해주세요.
load_dotenv()

if "OPENAI_API_KEY" not in os.environ:
    print("⚠️ OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인해주세요!")
else:
    print("✅ API 키가 성공적으로 로드되었습니다.")

✅ API 키가 성공적으로 로드되었습니다.


### 단계 1: Agent의 두 종류 이해하기
smol-agent에는 두 가지 주요 Agent 클래스가 있습니다.

#### CodeAgent 🤖 (프로그래머 Agent)

작동 방식: 도구를 사용하기 위해 Python 코드를 직접 생성하고 실행합니다.

장점: 반복문, 조건문 등 복잡한 로직을 수행할 수 있고, 여러 도구의 결과를 조합하는 등 유연성이 매우 높습니다.

단점: 코드를 직접 실행하므로 보안에 유의해야 합니다.

언제 사용?: 여러 단계를 거쳐야 하는 문제 해결, 데이터 처리, 동적인 작업이 필요할 때 적합합니다.

#### ToolCallingAgent 📞 (도구 사용 Agent)

작동 방식: 도구를 호출하기 위해 JSON 형식의 구조화된 데이터를 생성합니다.

장점: 정해진 형식으로만 도구를 호출하므로 매우 안정적이고 안전합니다.

단점: 코드를 실행할 수 없어 복잡한 로직이나 유연한 작업은 어렵습니다.

언제 사용?: 단순한 API 호출, 문서 조회 등 예측 가능한 작업을 안정적으로 수행할 때 적합합니다.

### CodeAgent 만들고 실행하기

피보나치 수열을 계산하는 간단한 임무를 통해 CodeAgent를 만들어 보겠습니다. 이 Agent는 Python 코드를 생성하여 계산을 수행할 것입니다.

In [3]:
from smolagents import CodeAgent, LiteLLMModel

# 1. Agent의 두뇌가 될 LLM 모델을 선택합니다.
#    'gpt-4o' 외에도 'gpt-4.1-mini' 등 다양한 모델을 쓸 수 있습니다.
model = LiteLLMModel(model_id="gpt-4.1-mini")

# 2. CodeAgent를 생성합니다.
#    - tools=[]: 아직 우리만의 도구는 없으므로 빈 리스트를 전달합니다.
#    - add_base_tools=True: 'smol-agent'가 기본으로 제공하는 도구(수학 계산 등)를 추가합니다.
agent = CodeAgent(
    model=model,
    tools=[], 
    add_base_tools=True,
)

print("✅ 나의 첫 CodeAgent가 생성되었습니다!")

✅ 나의 첫 CodeAgent가 생성되었습니다!


In [4]:
# Agent에게 피보나치 수열 계산을 요청합니다.
question = "피보나치 수열의 20번째 숫자는 무엇인가요?"
result = agent.run(question)

print("\n--- Agent의 최종 답변 ---")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 피보나치 수열의 20번째 숫자는 무엇인가요?                                                                       │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def fibonacci(n: int) -> int:                                                                                    
      if n == 0:                                                                                                   
          return 0                                                                                                 
      elif n == 1:                                                                                                 
          return 1                                                                                                 
      a, b = 0, 1                                                                                                  
      for _ in range(2, n+1):                                                                                      
          a, b = b, a + b                                                                                          
      return b                                                                                                     
                                                                                                                   
  fib_20 = fibonacci(20)                                                                                           
  final_answer(fib_20)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 6765

[Step 1: Duration 4.40 seconds| Input tokens: 2,001 | Output tokens: 130]


--- Agent의 최종 답변 ---
6765


### 나만의 커스텀 도구 만들기: "허깅페이스 인기 모델 찾기"

이번에는 Agent에게 새로운 능력을 부여해 보겠습니다. 특정 분야(task)에서 가장 많이 다운로드된 허깅페이스 모델을 찾아주는 도구를 직접 만들어 봅시다.

공식 문서에 나온 방법들 중, 가장 직관적인 @tool 데코레이터를 사용하는 방법으로 만들어보겠습니다.

In [6]:
from smolagents import tool
from huggingface_hub import list_models

@tool
def get_most_downloaded_model(task: str) -> str:
    """
    주어진 task에 대해 허깅페이스 허브에서 가장 많이 다운로드된 모델의 ID를 반환합니다.
    
    Args:
        task (str): "text-classification"이나 "text-to-video" 같은 모델의 task.
    """
    most_downloaded_model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
    return most_downloaded_model.id

print("✅ 'get_most_downloaded_model' 도구가 성공적으로 생성되었습니다!")

✅ 'get_most_downloaded_model' 도구가 성공적으로 생성되었습니다!


In [7]:
# 1. 새로운 도구를 장착한 Agent를 생성합니다.
tool_agent = CodeAgent(
    model=model,
    tools=[get_most_downloaded_model], # 직접 만든 도구를 리스트에 담아 전달!
)

# 2. Agent에게 새로운 도구를 사용해야만 해결할 수 있는 질문을 던집니다.
question = "허깅페이스 허브의 'text-to-image' task에서 가장 인기 있는 모델은 무엇인가요?"
result = tool_agent.run(question)

print("\n--- Agent의 최종 답변 ---")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 허깅페이스 허브의 'text-to-image' task에서 가장 인기 있는 모델은 무엇인가요?                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  most_popular_text_to_image_model = get_most_downloaded_model(task="text-to-image")                               
  final_answer(most_popular_text_to_image_model)                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: CompVis/stable-diffusion-v1-4

[Step 1: Duration 5.16 seconds| Input tokens: 1,971 | Output tokens: 93]


--- Agent의 최종 답변 ---
CompVis/stable-diffusion-v1-4


## 간단히 웹 검색 에이전트를 만들어 보고 싶다면?

Google search를 이용한 방법도 뒤에는 배울거지만, 가장 쉬운 사용 방법은 DuckDuckGoSearchTool를 이용하는 것.

In [9]:
from smolagents import CodeAgent, LiteLLMModel, DuckDuckGoSearchTool

model = LiteLLMModel(model_id="gpt-4.1-mini")
agent = CodeAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
)

# Now the agent can search the web!
result = agent.run("지금 서울 날씨는?")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 지금 서울 날씨는?                                                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_result = web_search(query="current weather in Seoul")                                                     
  print(search_result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Current 뜻과 예시 - 네이버 블로그](https://blog.naver.com/PostView.nhn?blogId=engram_blog&logNo=223314942024)
원어민들은 current를 어떻게 사용하나요? Current의 사용 예시는 아래와 같습니다. '현재의'일 때 The current situation 
requires immediate attention. (현재 상황은 즉각적인 주의를 필요로 한다.) We discussed the …

[current 뜻 - current 한국어 뜻 - iChaCha사전](https://ko.ichacha.net/english/current.html)
occurring in or belonging to the present time; "current events"; "the current topic"; "current negotiations"; 
"current psychoanalytic theories"; "the ship''s current position"

[CURRENT | 영어를 한국어로 번역: Cambridge 
Dictionary](https://dictionary.cambridge.org/ko/사전/영어-한국어/current)
CURRENT 번역: 현재의, 기류, 전류. Cambridge 영어-한국어 사전 에서 자세히 알아보기.

[current 뜻 - 영어 사전 | current 해석과 발음 및 문장 사용 예 - 
fatLingo](http://www.fatlingo.com/translate.php?set_id=11&section_id=25&lang=kor&word=current)
영어단어 current 의 뜻은 다음과 같습니다. 현재의. 사용 예와 해석은 아래와 같습니다.

[current - WordReference 영-한 사전](https://www.wordreference.com/enko/current)
구글 번역의 기계 번역을 볼 용어: current 다른 언어로: 스페인어 | 불어 | 이탈리아어 | 포르투갈어 | 루마니아어 | 
독일어 | 네덜란드어 | 스웨덴어 | 러시아어 | 폴란드어 | 체코어 | 그리스어 | 터키어 | 중국어 | 일본어 | 아랍어

[current의 다양한 뜻 - 네이버 블로그](https://m.blog.naver.com/ne5korea/206336008)
Mar 16, 2014 · ① 현대의, 지금의, 현행의 We enclosed our current price list with the letter. (현 시세표를 편지에 
동봉했습니다.) ② 통용되는 That word is no longer current in the United States. (그 말은 미국에서는 …

[current: 뜻과 사용법 살펴보기 | RedKiwi Words](https://redkiwiapp.com/ko/english-guide/words/current)
current [ˈkʌrənt] 라는 용어는 '현재 상황은 즉각적인 조치를 요구한다'와 같이 현재에 속하는 것을 가리킨다. 또한 '강의
흐름이 너무 강해서 헤엄칠 수 없었다'에서와 같이 물, 공기 또는 전기의 흐름을 설명할 수도 있습니다.

[Current 뜻과 예시 - Engram](https://blog-ko.engram.us/current-meaning/)
Jan 7, 2024 · Current는 맥락에 따라 형용사로는 '현재의', 명사로는 ' (물이나 공기의) 흐름', '전류', ' (사람들 
사이의) 경향'을 의미합니다. 각각의 자세한 의미와 예시는 아래와 같습니다.

[current 뜻 - 영어 사전 | current 의미 해석 - wordow.com](https://ko.wordow.com/english/dictionary/current)
에서 한국어 내부, 우리는 어떻게 설명 할current영어 단어 그것은? current영어 단어는 다음과 같은 의미를 한국어 :현재.
Meaning of current for the defined word. 문법적으로, 이 워드 "current" 는 형용사. 그것은 또한 명사, 좀 더 …

[current (【형용사】현재의, 지금의 ) 뜻, 용법, 그리고 예문 | Engoo 
Words](https://engoo.co.kr/app/words/word/current/zga0ELstQmCjlQAAAABfHQ)
"current"와 관련된 레슨 교재 The journal Current Biology published their findings. Current Biology 저널은 그들의 
연구 결과를 발표했다.

Out: None

[Step 1: Duration 2.91 seconds| Input tokens: 1,941 | Output tokens: 57]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Extracting the relevant weather info from the search result snippet                                            
  weather_summary = "Currently: 25 °C. More clouds than sun."                                                      
  final_answer(weather_summary)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Currently: 25 °C. More clouds than sun.

[Step 5: Duration 4.57 seconds| Input tokens: 18,944 | Output tokens: 393]

Currently: 25 °C. More clouds than sun.
